In [1]:
import torch
print(torch.cuda.is_available())  # Should print True if CUDA is available
print(torch.cuda.current_device())

True
0


In [2]:
torch.cuda.empty_cache()

In [17]:
!nvidia-smi

Wed Sep 25 18:58:58 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1080 Ti     On  |   00000000:81:00.0 Off |                  N/A |
| 41%   73C    P2             85W /  250W |   10043MiB /  11264MiB |     95%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [4]:
!kill -9 2450845

In [5]:
from transformers import LlamaTokenizer, pipeline, AutoModelForCausalLM

2024-09-25 18:13:57.649593: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-25 18:13:57.668890: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-25 18:13:57.674953: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-25 18:13:57.690609: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-25 18:13:58.721434: W tensorflow/compiler/tf2

In [6]:
from transformers import  AutoTokenizer

In [7]:
model_name = "mistralai/Mistral-7B-Instruct-v0.3"

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)

/upb/users/b/balram/profiles/unix/cs/.local/lib/python3.12/site-packages/transformers/models/auto/tokenization_auto.py:786: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [9]:
# Load the model
model = AutoModelForCausalLM.from_pretrained(model_name,device_map="auto")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.


In [10]:
# Create a pipeline for text generation
text_generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [16]:
# Generate text based on a prompt
prompt = "Who is Ada Lovelace?"
generated_text = text_generator(prompt, max_length=500)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [18]:
# print the result
print(generated_text[0]['generated_text'])

Who is Ada Lovelace?

Ada Lovelace was an English mathematician and writer, chiefly known for her work on Charles Babbage's early mechanical general-purpose computer, the Analytical Engine. Her notes on the engine include what is recognized as the first algorithm intended to be processed by a machine, and she is often regarded as the first computer programmer.

Lovelace was the daughter of the poet Lord Byron and his wife Anne Isabella Milbanke. She was named after her mother, Ada Byron, who was given the name Ada in honor of Countess Ada Lovelace, a friend of her grandmother. Lovelace was educated at home by various tutors, including Mary Somerville, and showed a talent for mathematics from an early age.

In 1833, Lovelace met Charles Babbage, who was working on the design of the Analytical Engine. Babbage was impressed by Lovelace's mathematical abilities and asked her to translate an article on the engine by Italian mathematician Luigi Menabrea. In the process of translating the art

In [19]:
import json

In [20]:
import jsonlines

In [21]:
import re

In [22]:
def load_prompts_data(filepath):
    with jsonlines.open(filepath) as reader:
        return list(reader)

In [23]:
def extract_prompt_info(item):
    prompt_id = item.get('id')
    prompt_text = item.get('prompt')
   #print("prompt_text",prompt_text)
    return prompt_id, prompt_text

In [24]:
# Assuming 'text_generator' is already defined somewhere in your environment
def generate_text(prompt_text, max_length=500, num_return_sequences=1):
    response = text_generator(prompt_text, max_length=max_length, num_return_sequences=num_return_sequences)
    return response

In [25]:
def extract_test_output(response):
    #print("response",response)
    if response and len(response) > 0:
        generated_text = response[0].get('generated_text', '')
        match = re.search(r'Test Output:\s*(.*?)\s*(?:\n|$)', generated_text, re.DOTALL)
        if match:
            return match.group(1).strip()
    return 'Output not found'

In [26]:
def parse_model_output(model_output):
    # Initialize an empty list to store triples
    triples = []
    
    # Remove leading/trailing whitespace and split the input data into lines
    lines = [line.strip() for line in model_output.strip().split('\n')]
    
    # Define the regex pattern to match triples in the format: relation(subject, object)
    pattern = re.compile(r'(.+?)\s*\(([^,]+),\s*([^)]+)\)')
    
    for line in lines:
        # Find all matches for the pattern in the line
        matches = pattern.findall(line)
        
        for match in matches:
            relation, subject, obj = match
            # Clean up subject and object values
            subject = subject.strip()
            obj = obj.strip()
            
            # Append the extracted triple to the list
            triples.append({"sub": subject, "rel": relation, "obj": obj})
    
    # Return the list of triples
    return triples

In [27]:
JSONL_FILEPATH = 'Wikidata/Input_Prompts/ont_10_culture_prompts.jsonl'
output_filepath='Wikidata/Response/ont_10_culture_llm_response.jsonl'

In [28]:
import json

def save_triples_to_jsonl(processed_data, output_filepath):
    """
    Save the processed triples into a JSONL file format.

    Args:
        processed_data (list): A list of dictionaries containing the 'id' and 'triples'.
        output_filepath (str): The output file path where the JSONL data should be saved.
    """
    with open(output_filepath, 'w', encoding='utf-8') as outfile:
        for entry in processed_data:
            jsonl_entry = {
                "id": entry["id"],
                "triples": entry["triples"]
            }
            # Convert dictionary to a JSON string and write it to the file
            outfile.write(json.dumps(jsonl_entry, ensure_ascii=False) + "\n")

In [29]:
def main(filepath, output_filepath, num_prompts=4):
    prompts_data = load_prompts_data(filepath)
    processed_data = []
    for i in range(min(num_prompts, len(prompts_data))):
        item = prompts_data[i]
        prompt_id, prompt_text = extract_prompt_info(item)
        response = generate_text(prompt_text)
        test_output = extract_test_output(response)
        
        # Debugging: Print the test output to ensure it's correctly extracted
        print(f"Test Output for ID {prompt_id}: {test_output}")
        
        # Parse the test output into triples
        triples = parse_model_output(test_output)
        
        # Debugging: Print the parsed triples to ensure they're correct
        print(f"Parsed Triples for ID {prompt_id}: {triples}")
        
        processed_entry = {
            "id": prompt_id,
            "triples": triples
        }
        
        processed_data.append(processed_entry)
    
    # Save the processed data into a new jsonl file
    save_triples_to_jsonl(processed_data, output_filepath)
    print(f"Processed triples saved to {output_filepath}")

In [ ]:
# Replace 'your_input_filepath.jsonl' with your actual input file path
# Replace 'your_output_filepath.jsonl' with your desired output file path
main(JSONL_FILEPATH, output_filepath, num_prompts=1)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
